In [1]:
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [58]:
df = pd.read_csv("test_data.csv")
df.shape

(1048575, 6)

# 1만 데이터

In [59]:
df = df[:10000]
df['sam'] = 0

In [51]:
%%timeit
#단순 반복문
for r in range(len(df)):
    df.loc[r,'sam'] = df.loc[r,'accu_cv']+df.loc[r,'term_cv']

2.48 s ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%%timeit
#itterrows
new = []
for i,r in df.iterrows():
    new.append(r['accu_cv']+r['term_cv'])
df['sam'] = new

809 ms ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit
#apply
df['sam'] = df.apply(lambda r : r['accu_cv']+r['term_cv'], axis=1)

94.2 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
%%timeit
#벡터화 연산
df['sam'] = df['accu_cv']+df['term_cv']

206 µs ± 7.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# 100만 데이터

100만에서 단순 반복문은 사용 불가(너무 오래걸림)

In [34]:
#100만 데이터
df = pd.read_csv("test_data.csv")
df = df[:1000000]
df['sam'] = 0
df.shape

(1000000, 7)

In [35]:
%%time
#itterrows
new = []
for i,r in df.iterrows():
    new.append(r['accu_cv']+r['term_cv'])
df['sam'] = new

Wall time: 1min 19s


In [36]:
%%time
#apply
df['sam'] = df.apply(lambda r : r['accu_cv']+r['term_cv'], axis=1)

Wall time: 9.96 s


In [37]:
%%time
#벡터화 연산
df['sam'] = df['accu_cv']+df['term_cv']

Wall time: 7.82 ms


# 조건부 연산

In [132]:
df = pd.read_csv("test_data.csv")
df['sam'] = 0
df.shape

(1048575, 7)

In [65]:
df['sam'].value_counts()

0    1048575
Name: sam, dtype: int64

In [88]:
%%time
#조건 하나
df.loc[df['accu_cv']>0.5,'sam'] = 1
df['sam'].value_counts()

Wall time: 36.2 ms


1    1014345
0      34230
Name: sam, dtype: int64

In [90]:
%%time
#다중 조건
df.loc[(df['accu_cv']>0.5) & (df['term_cv']!=0),'sam'] = 1
df['sam'].value_counts()

Wall time: 38.1 ms


1    687629
0    360946
Name: sam, dtype: int64

# 조건문이 필요한 경우

조건문이 필요할 경우 벡터연산이 힘들어질 수 있다<br/>
속도는 느리지만 apply 사용이 편함

In [177]:
df = pd.read_csv("test_data.csv")
df['sam'] = 0
df.shape

(1048575, 7)

In [104]:
%%time
#if else
df['sam'] = df.apply(lambda r : 0 if r['accu_cv']<0.5 else(1 if r['term_cv']!=0 else 2), axis=1)
df['sam'].value_counts()

Wall time: 10.2 s


1    720818
2    326716
0      1041
Name: sam, dtype: int64

In [144]:
df.loc[df['accu_cv']<0.5, 'sam'] = 1
df['sam'].value_counts()

0    1047534
1       1041
Name: sam, dtype: int64

In [146]:
df.loc[df['accu_cv']>=0.5, 'sam'] = 1
df['sam'].value_counts()

1    1014345
0      34230
Name: sam, dtype: int64

위의 경우 accu_cv의 값이<br/>
0.5보다 작은것(< 0.5)은 1041개이지만<br/>
0.5보다 크거나 같은 것이 아닌것(>= 0.5가 아닌것)은 34230개로 다른 결과가 나온다


이는 빈 데이터(NaN값)에 의해 발생하는 문제로 이러한 문제에 조건문이 대응하기 편하기 때문에 편의에 따라 적절한 방법을 이용하는게 좋다

In [178]:
#accu_cv에 포함되어있는 빈 데이터의 개수는 33189개로 1041과 34230의 차이가 33189로 해당 문제임을 알 수 있다
len(df.loc[df['accu_cv'].isnull()])

33189